In [89]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import numpy as np

import torch.autograd as autograd
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from collections import Counter

## Character-level LSTM

In [4]:
training_data = [("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
                ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])]

In [9]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
print(word_to_ix)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [10]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

ix_to_tag = {ix:tag for tag,ix in tag_to_ix.items()}
ix_to_word = {ix:word for word,ix in word_to_ix.items()}

In [2]:
def sent_to_char(sent):
    char_list = []
    for word in sent:
        word_chars = []
        for character in word:
            word_chars.append(character)
        char_list.append(word_chars)
    return char_list

In [5]:
char_to_ix = {}
for sent, tags in training_data:
    for word in sent_to_char(sent):
        for char in word:
            if char not in char_to_ix:
                char_to_ix[char] = len(char_to_ix)

In [6]:
sent_to_char(['a','sentence'])

[['a'], ['s', 'e', 'n', 't', 'e', 'n', 'c', 'e']]

In [19]:
class char_lstm(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, vocab_size_char, tagset_size):
        super(char_lstm, self).__init__()
        self.hidden_dim = hidden_dim

        self.char_embeddings = nn.Embedding(vocab_size_char, embedding_dim)
        self.lstm_char = nn.LSTM(embedding_dim, hidden_dim)
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim + hidden_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
                
        self.hidden = self.init_hidden(self.hidden_dim)
        self.hidden_char = self.init_hidden(self.hidden_dim)
        

    def forward(self, sentence_in, char_in):
        num_words = sentence_in.shape[0]
        embeds = self.word_embeddings(sentence_in)
                
        char_output_list = []
        for word_chars in char_in:
            
            ## reset the hidden layer to remove make the representation for consecutive words independent
            self.hidden_char = self.init_hidden(self.hidden_dim)
            char_emb = self.char_embeddings(word_chars)
            lstm_out_1, self.hidden_char = self.lstm_char(char_emb.view(char_emb.shape[0], 1, -1),self.hidden_char)        
            
            ## this keeps only the last output from each charcter sequence
            char_output_list.append(lstm_out_1[-1])

        char_output_tensor = torch.cat(char_output_list)

        merged = torch.cat([embeds, char_output_tensor], dim=1) 
        
        self.hidden = self.init_hidden(self.hidden_dim)
        
        lstm_out_2, self.hidden = self.lstm(merged.view(num_words, 1, -1),self.hidden)
        lstm_out_2 = lstm_out_2.view(num_words,-1)
        
        tag_scores = self.hidden2tag(lstm_out_2)        
        tag_scores = F.log_softmax(tag_scores, dim=1)
        
        return tag_scores
    
    def init_hidden(self,size):
        return (torch.zeros(1, 1, size),torch.zeros(1, 1, size))

In [20]:
EMBEDDING_DIM,HIDDEN_DIM = 6,3
model = char_lstm(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(char_to_ix), len(tag_to_ix))

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [21]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def prepare_sequence_char(seq, to_ix):
    sentence_tensor = []
    for word in seq:
        idxs = [to_ix[w] for w in word]
        word_tensor = torch.tensor(idxs, dtype=torch.long)
        sentence_tensor.append(word_tensor)
    return sentence_tensor

In [22]:
## Try a forward pass
for sentence, tags in training_data:
    sentence_in = prepare_sequence(sentence, word_to_ix)
    char_in = prepare_sequence_char(sent_to_char(sentence), char_to_ix)
    targets = prepare_sequence(tags, tag_to_ix)
    tag_scores = model(sentence_in,char_in)
    break

In [23]:
tag_scores

tensor([[-0.6591, -1.4407, -1.4028],
        [-0.5873, -1.6631, -1.3680],
        [-0.5987, -1.6975, -1.3193],
        [-0.5825, -1.6979, -1.3531],
        [-0.6824, -1.5441, -1.2690]], grad_fn=<LogSoftmaxBackward>)

## Train Model

In [24]:
EMBEDDING_DIM,HIDDEN_DIM = 6,4
model = char_lstm(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(char_to_ix), len(tag_to_ix))

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in trange(100):
    model.zero_grad()
    
    for sentence, tags in training_data:
        
        sentence_in = prepare_sequence(sentence, word_to_ix)
        char_in = prepare_sequence_char(sent_to_char(sentence), char_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in,char_in)

        loss = loss_function(tag_scores, targets)
        
        loss.backward()
        optimizer.step()

In [25]:
# See what the scores are after training
with torch.no_grad():
    for sentence, tags in training_data:
        sentence_in = prepare_sequence(sentence, word_to_ix)
        char_in = prepare_sequence_char(sent_to_char(sentence), char_to_ix)
        tag_scores = model(sentence_in,char_in)

        preds_ix = np.argmax(tag_scores.detach().numpy(),axis=1)
        preds = [ix_to_tag[p] for p in preds_ix]
        print(sentence)
        
        print(f'Preds: {preds}')
        print(f'Tags:  {tags}')
        
        print(f'-------------------------------')

['The', 'dog', 'ate', 'the', 'apple']
Preds: ['DET', 'NN', 'V', 'DET', 'NN']
Tags:  ['DET', 'NN', 'V', 'DET', 'NN']
-------------------------------
['Everybody', 'read', 'that', 'book']
Preds: ['NN', 'V', 'DET', 'NN']
Tags:  ['NN', 'V', 'DET', 'NN']
-------------------------------
